In [35]:
import pandas as pd

In [36]:
df = pd.read_parquet("../data/builds/dx_occurrence_test/output/df_input.parquet")
df.head()

,design_version,group,position,process,variable,value,date,parameter,value_norm,order,year,month,day,hour,minute,design_version_position,layer,occurrence,process_id,step
0,426816_B,CUEX_13,40.0,Laser,las_11,NaN,NaT,Pulsenergie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
1,426816_B,CUEX_13,40.0,Laser,las_12,NaN,NaT,SkalierungX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
2,426816_B,CUEX_13,40.0,Laser,las_13,NaN,NaT,SkalierungY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
3,426816_B,CUEX_13,40.0,Laser,las_14,NaN,NaT,Laserpower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
4,426816_B,CUEX_13,40.0,Laser,las_15,NaN,NaT,T_Drv0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0


In [45]:
# ASSIGN STEPS
import numpy as np
df_processed = df
steps_dict = {}
for pro in df_processed["process"].unique():
    for occ in df_processed.set_index("process").loc[pro, "occurrence"].unique():
        mask = np.logical_and(df_processed["process"] == pro, df_processed["occurrence"] == occ)
        steps = df_processed.set_index(["process","occurrence"]).loc[pro].loc[occ]["position"].unique()
        steps_map = {step:i+1 for i,step in enumerate(steps)}
        steps_dict[str(pro)+"_"+str(occ)] = steps_map
        df_processed.loc[mask, "step"] = df_processed.loc[mask, "position"].map(steps_map)

In [46]:
df_processed

,design_version,group,position,process,variable,value,date,parameter,value_norm,order,year,month,day,hour,minute,design_version_position,layer,occurrence,process_id,step
0,426816_B,CUEX_13,40.0,Laser,las_11,NaN,NaT,Pulsenergie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
1,426816_B,CUEX_13,40.0,Laser,las_12,NaN,NaT,SkalierungX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
2,426816_B,CUEX_13,40.0,Laser,las_13,NaN,NaT,SkalierungY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
3,426816_B,CUEX_13,40.0,Laser,las_14,NaN,NaT,Laserpower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
4,426816_B,CUEX_13,40.0,Laser,las_15,NaN,NaT,T_Drv0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426816_B_40,"3,4",1,Laser_1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2152875,453828_B,CXOL_8,680.0,Microetch,mic_51,0.000000,2023-11-30 06:37:52,G_M5_Conductivity_AV Minimum,0.000000,13.0,2023.0,11.0,30.0,6.0,37.0,453828_B_680,Endoberfläche,4,Microetch_4,2.0
2152876,453828_B,CXOL_8,680.0,Microetch,mic_52,0.107212,2023-11-30 06:37:52,G_M5_Conductivity_AV Maximum,0.107212,13.0,2023.0,11.0,30.0,6.0,37.0,453828_B_680,Endoberfläche,4,Microetch_4,2.0
2152877,453828_B,CXOL_8,680.0,Microetch,mic_53,0.272100,2023-11-30 06:37:52,G_M5_Conductivity_AV Std.Dev.,0.272100,13.0,2023.0,11.0,30.0,6.0,37.0,453828_B_680,Endoberfläche,4,Microetch_4,2.0
2152878,453828_B,CXOL_8,680.0,Microetch,mic_70,0.500000,2023-11-30 06:37:52,Val_Spüle Bakterienbefall-M5.4,1.000000,13.0,2023.0,11.0,30.0,6.0,37.0,453828_B_680,Endoberfläche,4,Microetch_4,2.0


In [23]:
pivot = df.pivot_table(
    index='position', 
    columns='process_id', 
    aggfunc='size', 
    fill_value=0
).sort_index()
print(pivot)

process_id  Galvanic_1  Galvanic_2  Galvanic_3  Laser_1  Laser_2  Laser_3  \
position                                                                    
40.0                 0           0           0    62928        0        0   
60.0                 0           0           0        0        0        0   
70.0            338590           0           0        0        0        0   
90.0             62230           0           0        0        0        0   
100.0                0           0           0       72        0        0   
180.0                0           0           0        0        0        0   
200.0                0           0           0        0        0        0   
230.0                0           0           0        0    52704        0   
240.0                0           0           0        0    54144        0   
250.0                0           0           0        0     9252        0   
260.0                0           0           0        0     9180        0   

In [40]:
pivot = df.pivot_table(
    index=['process', 'occurrence', 'step'], 
    columns='process', 
    aggfunc='size', 
    fill_value=0
).sort_index()

print(pivot)

process                    Galvanic  Laser  Microetch  Multibond  Plasma
process   occurrence step                                               
Galvanic  1          1.0     338590      0          0          0       0
                     2.0      62230      0          0          0       0
          2          1.0     248185      0          0          0       0
                     2.0      51205      0          0          0       0
          3          1.0     283220      0          0          0       0
                     2.0      56105      0          0          0       0
Laser     1          1.0          0  62928          0          0       0
                     2.0          0     72          0          0       0
          2          1.0          0  52704          0          0       0
                     2.0          0  54144          0          0       0
                     3.0          0   9252          0          0       0
                     4.0          0   9180         